In [1]:
#%run B_equalize-histograms-laggui.ipynb
#%run B_equalize-histograms-mask.ipynb
%run B_equalize-histograms-sklearn.ipynb

len(fnames)=684


In [2]:
ttl.args

Namespace(tag='2021-02-21_TimeTeleScope', date='2021-02-21', in_path='../orchid', cache_path='cache_path', videos_path='videos', figwidth=15, ds=1, fps=12, verb=0)

# processing the full stack into a movie

Now that we know how to match the images from the stack to some reference, we can process the whole stack and save the matched images into a cache folder:


In [3]:
fnames = ttl.glob()

In [4]:
frames = []
n_bits = 16 # precision for saving images

for i, fname_new in enumerate(fnames):
    # print(f'{fname_new=}')
    image_matched = histMatch(ttl.process(fname_new), image_reference)
    #print(image_matched.min(), image_matched.max())
    image_matched *= 2**n_bits-1
    image_matched = image_matched.astype(np.uint16)

    fname = f'{ttl.args.cache_path}/{ttl.args.tag}_{i}.png'
    
    import cv2
    cv2.imwrite(fname, cv2.cvtColor(image_matched, cv2.COLOR_RGB2BGR))

    frames.append(fname)
    

## Merging single frames 

We use the excellent [moviepy](https://pypi.org/project/moviepy/) library to convert the single frames into a movie:

In [5]:
import moviepy.editor as mpy
import moviepy.video.io.ImageSequenceClip
def make_mp4(moviename, frames, fps, do_delete=True):
    clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(frames, fps=fps)
    clip.write_videofile(moviename)
    if do_delete: 
        for frame in frames: os.remove(frame)
    return moviename

video_name = f'{ttl.args.videos_path}/{ttl.args.tag}.mp4'
make_mp4(video_name, frames, ttl.args.fps, do_delete=False)

t:   0%|          | 2/685 [00:00<00:36, 18.53it/s, now=None]

Moviepy - Building video videos/2021-02-21_TimeTeleScope.mp4.
Moviepy - Writing video videos/2021-02-21_TimeTeleScope.mp4



Moviepy - Done !
Moviepy - video ready videos/2021-02-21_TimeTeleScope.mp4


'videos/2021-02-21_TimeTeleScope.mp4'

Eventually, we can have a preview using:

In [6]:
#from IPython.display import Image, display
width = 1024

def show(filename, width=1024):
    from IPython.display import Video
    # https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html?highlight=IPython.display#IPython.display.Video
    return Video(filename, html_attributes=f"controls muted autoplay width={width}")

# show(video_name, width=1024)

In [7]:
if verb:

    image_matched = histMatch(ttl.process(image_new), ttl.process(image_reference))
    fig, ax = plt.subplots(figsize=ttl.figsize)
    ax.imshow(image_matched, vmin=0, vmax=255);

    image_matched.min(), image_matched.max()

Moreover, it's easy to generate a gif (with some downscaling to avoid having a huge file...):

In [8]:
def make_gif(gifname, frames, fps, ds=1, tds=1, do_delete=True):
    import imageio
    with imageio.get_writer(gifname, mode='I', fps=fps) as writer:
        for frame in frames[::tds]:
            img = imageio.imread(frame)
            if ds > 1:
                nx, ny, three = img.shape
                # CAREFUL ! for a strange reason openCV inverts H and W ??
                img = cv2.resize(img, (ny//ds, nx//ds),
                                interpolation = cv2.INTER_AREA)
                nx, ny, three = img.shape            
            writer.append_data(img)

    from pygifsicle import optimize
    optimize(str(gifname))
    if do_delete: 
        for frame in frames: os.remove(fname)
    return gifname

In [9]:
gif_name = f'{ttl.args.videos_path}/{ttl.args.tag}.gif'
make_gif(gif_name, frames, ttl.args.fps, ds=8, tds=4, do_delete=False)

'videos/2021-02-21_TimeTeleScope.gif'

Et voilà!

## some book keeping for the notebook

In [10]:
%load_ext watermark
%watermark -i -h -m -v -p numpy,cv2,moviepy,skimage,matplotlib  -r -g -b

Python implementation: CPython
Python version       : 3.9.1
IPython version      : 7.19.0

numpy     : 1.20.1
cv2       : 4.5.1
moviepy   : 1.0.3
skimage   : 0.18.1
matplotlib: 3.3.3

Compiler    : Clang 12.0.0 (clang-1200.0.32.29)
OS          : Darwin
Release     : 20.2.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

Hostname: ekla

Git hash: 306725c1778bf4d3cb07172966b8af9432172dee

Git repo: https://github.com/laurentperrinet/TimeTeleScope

Git branch: main

